# **TensorFlow Project: Audiobooks**

**Extract the data from the csv**

In [2]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('sample_data/Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

**unscaled_inputs_all** [:,1:-1] 表示取所有行，去掉第一列（通常是 ID），去掉最后一列（标签 target），只留下中间的特征。所以 unscaled_inputs_all 就是 模型的输入数据。

**targets_all** 提取最后一列作为 标签 (target)，也就是客户是否会再次购买有声书：1 = 会购买 0 = 不会购买

In [3]:
unscaled_inputs_all

array([[1620.  , 1620.  ,   19.73, ..., 1603.8 ,    5.  ,   92.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,    0.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,  388.  ],
       ...,
       [2160.  , 2160.  ,    6.14, ...,    0.  ,    0.  ,    0.  ],
       [1620.  , 1620.  ,    5.33, ...,  615.6 ,    0.  ,   90.  ],
       [1674.  , 3348.  ,    5.33, ...,    0.  ,    0.  ,    0.  ]])

In [17]:
targets_all.shape[0]

14084

**Balance the dataset**

In [5]:
num_one_targets = int(np.sum(targets_all)) # 计算所有 target 里 1 的数量，有多少个1（有多少人会买）
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

**for loop(遍历所有样本)**

如果当前行的 target = 0，就把 zero_targets_counter 加一。

如果 0 的数量超过了 1 的数量，就把该行的索引记录下来。

这样做的目的：
⚡ 让 0 的数量不要比 1 多，即保证 0 和 1 的样本数量一致。(平衡购买和不购买的人)

**np.delete** 用delete()把数据里多余出来的0删除掉，这样让数据里1的数量和0的数量一样，得到一个 正负样本平衡的训练集。

在你这个 **Audiobooks 数据集里**，标签是：

0 = 用户不会再买

1 = 用户会再买

原始分布是 **不平衡的（约 5:1），** 大部分人都不会再买。

**为什么要平衡？**

如果不做平衡，模型会学出一个很“偷懒”的策略：

👉 只要预测所有人都 不会买 (0)，准确率就有 84%（因为 11,847/14,084 ≈ 0.84）。

看似准确率很高，但实际上模型 没**有学到真正有价值的规律。**

你最想预测的是：**哪些人会再买 (1)**，因为这是公司的盈利点。

**如果 1 的样本太少**，模型在训练中会“忽略”它们，导致对正样本（会买的人）预测很差。

**Standardize the inputs**

In [6]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
# 缩放数据 ex:（-1 0 1）

**Shuffle the data**

In [7]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

**np.arange()** 会创建一个从 0 到 N-1 的数组，其中 N 是样本的总数。例如，如果有 4474 个样本：[0, 1, 2, 3, 4, ..., 4473]

**np.random.shuffle()** 随机打乱这些索引

**shuffled_inputs shuffled_targets** 把打乱的索引放进这两个数组里面，这样就把inputs和targets一起 随机打乱 了，防止模型学习到数据集的顺序模式。

**Split the dataset into train, validation, and test**

In [8]:
sample_count = shuffled_inputs.shape[0] # shape[0]是算出number of the element

train_sample_count = int(0.8*sample_count)
validation_sample_count = int(0.1*sample_count)
test_sample_count = sample_count - train_sample_count - validation_sample_count

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count + validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count + validation_sample_count]

test_inputs = shuffled_inputs[train_sample_count + validation_sample_count:]
test_targets = shuffled_targets[train_sample_count + validation_sample_count:]

print(np.sum(train_targets), train_sample_count, np.sum(train_targets) / train_sample_count)
print(np.sum(validation_targets), validation_sample_count, np.sum(validation_targets) / validation_sample_count)
print(np.sum(test_targets), test_sample_count, np.sum(test_targets) / test_sample_count)

1800.0 3579 0.5029337803855826
209.0 447 0.46756152125279643
228.0 448 0.5089285714285714


按常见比例划分：

训练集 (Train)：80%（number of the element * 0.8）

验证集 (Validation)：10%（number of the element * 0.1）

测试集 (Test)：10% (number of the element - 训练集 - 验证集)

**np.sum(train_targets)** 标签为 1 的样本数量

**train_sample_count** 该集合的总样本数

**np.sum(train_targets) / train_sample_count** 0和1 的比例

In [9]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

Save the three datasets in .npz

# **Create the machine learning algorithm**

**Import the relevant libraries**

In [10]:
import numpy as np
import tensorflow as tf

**Data**

In [11]:
npz_train = np.load('Audiobooks_data_train.npz')

train_inputs = npz_train['inputs'].astype(np.float64)
train_targets = npz_train['targets'].astype(np.int64)

npz_validation = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz_validation['inputs'].astype(np.float64), npz_validation['targets'].astype(np.int64)

npz_test = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz_test['inputs'].astype(np.float64), npz_test['targets'].astype(np.int64)

重新加载训练集文件，并把类型转换：

**float64 用于输入特征（神经网络输入必须是浮点数）**

**int64 用于标签（分类任务的类别编号）**

# **Model**

**outline, optimizers, loss, stopping and training**

In [12]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

输入层有 10 个特征(data feature)

输出层有 2 个类别（会买 / 不会买）

隐藏层设为 50 个神经元

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**optimizer='adam'**
→ 自适应学习率优化器（训练速度快，效果稳定）

**loss='sparse_categorical_crossentropy'**
→ 适合目标是整数标签（0 和 1）的分类任务(classification)。

**metrics=['accuracy']**
→ 训练时输出准确率。

In [14]:
batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose=2)

Epoch 1/100
36/36 - 2s - 47ms/step - accuracy: 0.6859 - loss: 0.5821 - val_accuracy: 0.7338 - val_loss: 0.5030
Epoch 2/100
36/36 - 0s - 5ms/step - accuracy: 0.7597 - loss: 0.4767 - val_accuracy: 0.7539 - val_loss: 0.4417
Epoch 3/100
36/36 - 0s - 5ms/step - accuracy: 0.7706 - loss: 0.4283 - val_accuracy: 0.7740 - val_loss: 0.4102
Epoch 4/100
36/36 - 0s - 5ms/step - accuracy: 0.7829 - loss: 0.4002 - val_accuracy: 0.7696 - val_loss: 0.3846
Epoch 5/100
36/36 - 0s - 4ms/step - accuracy: 0.7958 - loss: 0.3824 - val_accuracy: 0.7718 - val_loss: 0.3733
Epoch 6/100
36/36 - 0s - 5ms/step - accuracy: 0.7930 - loss: 0.3704 - val_accuracy: 0.7875 - val_loss: 0.3697
Epoch 7/100
36/36 - 0s - 5ms/step - accuracy: 0.8033 - loss: 0.3638 - val_accuracy: 0.7830 - val_loss: 0.3617
Epoch 8/100
36/36 - 0s - 6ms/step - accuracy: 0.8075 - loss: 0.3568 - val_accuracy: 0.7897 - val_loss: 0.3566
Epoch 9/100
36/36 - 0s - 8ms/step - accuracy: 0.8064 - loss: 0.3519 - val_accuracy: 0.7897 - val_loss: 0.3538
Epoch 10/

**batch_size = 100** 每个批次 100 条样本
**max_epochs = 100** 最多训练 100 轮

**model.fit()** 训练数据

**early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)**

如果验证集连续 2 轮不再提升，就提前停止（避免过拟合）。

**verbose=2** 每个 epoch 打印一行日志。


**callbacks（回调函数）**  是一种机制，
可以在模型训练的不同阶段（比如每个 epoch 结束时）执行一些额外操作，比如：

保存模型（ModelCheckpoint）

动态调整学习率（ReduceLROnPlateau）

提前停止训练（EarlyStopping）

**patience=2** 连续 2 次没提升后才停止

batch_size（批大小）

每次训练更新权重时用多少样本。

| 常见取值                 | 特点                  |
| -------------------- | ------------------- |
| **16 / 32**          | 小批次，噪声大但泛化好（适合小数据集） |
| **64 / 128**         | 常见默认值，兼顾速度与稳定性      |
| **256 / 512 / 1024** | 大批次，收敛快但可能过拟合（需大显存） |


max_epochs（最大轮数）

指整个数据集会被训练多少次。

| 常见取值       | 适用场景                            |
| ---------- | ------------------------------- |
| **10–20**  | 简单模型（如逻辑回归、浅层网络）                |
| **50–100** | 一般深度学习模型（CNN, RNN）              |
| **>200**   | 复杂模型（ResNet、Transformer等）或小学习率时 |


**Test the model**

In [15]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8605 - loss: 0.3281 


**model.evaluate()** 测试数据

In [16]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


Test loss: 0.33. Test accuracy: 83.71%
